### Import Libraries

In [1]:
import gensim.downloader
import pandas as pd
import numpy as np
import torch
import nltk
import string
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import random

### Set seed for reproducibility

In [2]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

### Download pretrained word embeddings

In [3]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

In [4]:
vocab = list(word2vec.index_to_key)
vocab_size = len(word2vec.index_to_key)
embedding_dim = 300

### Import Dataset

In [5]:
train_df = pd.read_csv('dataset part 2/train.csv')
test_df = pd.read_csv("dataset part 2/test.csv")
train_df.drop(columns=['label-fine'], inplace=True)
test_df.drop(columns=['label-fine'], inplace=True)

In [6]:
train_df

,label-coarse,text
0,0,How did serfdom develop in and then leave Russ...
1,1,What films featured the character Popeye Doyle ?
2,0,How can I find a list of celebrities ' real na...
3,1,What fowl grabs the spotlight after the Chines...
4,2,What is the full form of .com ?
...,...,...
5447,1,What 's the shape of a camel 's spine ?
5448,1,What type of currency is used in China ?
5449,4,What is the temperature today ?
5450,4,What is the temperature for cooking ?


Form a development set from a random subset (containing 500 examples) within the original training data. Remove these examples from original training file

In [7]:
development_df = train_df.sample(n=500, random_state=0) # randomly sample 500 rows from train df
train_df = train_df.drop(development_df.index) # remove sampled rows from train df

In [8]:
development_df

,label-coarse,text
2755,4,How many trees go into paper making in a year ?
3326,1,What concerts are held in New York this week ?
2204,5,Where did the sport of caber-tossing originate ?
2888,3,What kind of people took part in Shays ' Rebel...
2812,0,How is cologne made ?
...,...,...
5335,5,Where was George Washington born ?
1275,4,In what year did the Bounty mutiny happen ?
4508,0,What is the difference between khaki and chino ?
402,4,How many types of dogs ' tails are there - three


Randomly select 4 classes from 6 coarse labels, combine remaining 2 to form a single class 'OTHERS'. Adjust original data such that label for each sentence is updated accordingly

In [9]:
# Selected 2 labels for combining
combined_1 = 4
combined_2 = 5

# train_df['label-coarse'] = train_df['label-coarse'].astype(object)
# test_df['label-coarse'] = train_df['label-coarse'].astype(object)
# development_df['label-coarse'] = train_df['label-coarse'].astype(object)

# Assign label 5 as 4 as the OTHERS label
train_df.loc[(train_df['label-coarse'] == combined_2), 'label-coarse'] = combined_1
test_df.loc[(test_df['label-coarse'] == combined_2), 'label-coarse'] = combined_1
development_df.loc[(development_df['label-coarse'] == combined_2), 'label-coarse'] = combined_1

### Tokenize and remove punctuation

In [10]:
from nltk.tokenize import word_tokenize

def tokenize_and_remove_punctuation(text):
    # Create a translation table to replace punctuation with empty strings
    translator = str.maketrans('', '', string.punctuation)
    # Remove punctuation and replace with ''
    text_ = text.translate(translator)
    tokens = word_tokenize(text_)
    # filtered_tokens = [token for token in tokens if token in vocab]
    return tokens

In [11]:
train_df['text'] = train_df['text'].apply(lambda x: tokenize_and_remove_punctuation(x))
test_df['text'] = test_df['text'].apply(lambda x: tokenize_and_remove_punctuation(x))
development_df['text'] = development_df['text'].apply(lambda x: tokenize_and_remove_punctuation(x))

### Padding of Data for LSTM

In [12]:
# def pad_tokens(tokens, max_length, padding_value='<pad>'):
#     if len(tokens) < max_length:
#         tokens.extend([padding_value] * (max_length - len(tokens)))
#     return tokens

In [13]:
# max_length = max(train_df['text'].apply(len))

In [14]:
# train_df['text'] = train_df['text'].apply(pad_tokens, max_length=max_length)
# test_df['text'] = test_df['text'].apply(pad_tokens, max_length=max_length)
# development_df['text'] = development_df['text'].apply(pad_tokens, max_length=max_length)

### Get embedding matrix for embedding layer

In [15]:
embedding_matrix = word2vec.vectors

In [16]:
embedding_matrix.shape

(3000000, 300)

### Last element a zero array for out of vocab words

In [17]:
zero_array = np.zeros((300,))
embedding_matrix = np.vstack((embedding_matrix, zero_array)) # last element a zero array for OOV words

### Encode text in dataframe to indices

In [18]:
def tokens_to_indices(tokens, model):
    indices = []
    for token in tokens:
        try:
            indices.append(model.get_index(token))
        except:
            indices.append(len(embedding_matrix)-1)  # Handle out-of-vocabulary words
    return indices

In [19]:
train_df['word_indices'] = train_df['text'].apply(lambda x: tokens_to_indices(x, word2vec))
test_df['word_indices'] = test_df['text'].apply(lambda x: tokens_to_indices(x, word2vec))
development_df['word_indices'] = development_df['text'].apply(lambda x: tokens_to_indices(x, word2vec))

In [20]:
train_df

,label-coarse,text,word_indices
0,0,"[How, did, serfdom, develop, in, and, then, le...","[1190, 92, 178836, 1306, 1, 3000000, 145, 785,..."
1,1,"[What, films, featured, the, character, Popeye...","[470, 2485, 2723, 11, 1980, 65760, 8347]"
2,0,"[How, can, I, find, a, list, of, celebrities, ...","[1190, 50, 20, 359, 3000000, 711, 3000000, 666..."
3,1,"[What, fowl, grabs, the, spotlight, after, the...","[470, 43119, 10673, 11, 6236, 55, 11, 1035, 16..."
4,2,"[What, is, the, full, form, of, com]","[470, 4, 11, 335, 815, 3000000, 18117]"
...,...,...,...
5447,1,"[What, s, the, shape, of, a, camel, s, spine]","[470, 1280, 11, 2790, 3000000, 3000000, 28380,..."
5448,1,"[What, type, of, currency, is, used, in, China]","[470, 1474, 3000000, 2325, 4, 233, 1, 367]"
5449,4,"[What, is, the, temperature, today]","[470, 4, 11, 4360, 205]"
5450,4,"[What, is, the, temperature, for, cooking]","[470, 4, 11, 4360, 2, 5195]"


### Create custom dataset, collate function for dataloader and dataloaders

In [21]:
import torch
from torch.utils.data import Dataset

class MyCustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [22]:
def collate_fn(batch):
    data, targets = zip(*batch)
    
    # Sort the batch by sequence length (optional, but can improve efficiency)
    sorted_indices = sorted(range(len(data)), key=lambda i: len(data[i]), reverse=True)
    data = [data[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]

    # Create a list of sequences and their corresponding lengths
    sequences = [torch.tensor(seq) for seq in data]
    lengths = [len(seq) for seq in sequences]

    # Pad the sequences to the length of the longest sequence in the batch
    padded_sequences = pad_sequence(sequences, batch_first=True)

    # Create packed sequence for RNNs (optional, if you're using an RNN)
    # packed_sequences = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)

    return padded_sequences, torch.tensor(targets), torch.tensor(lengths)

In [23]:
train_dataset = MyCustomDataset(train_df['word_indices'].to_numpy(), train_df['label-coarse'].to_numpy())
test_dataset = MyCustomDataset(test_df['word_indices'].to_numpy(), test_df['label-coarse'].to_numpy())
development_dataset = MyCustomDataset(development_df['word_indices'].to_numpy(), development_df['label-coarse'].to_numpy())

In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
development_dataloader = DataLoader(development_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

### LSTM Text Classifier Architecture

In [25]:
class LSTMTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, device, aggregate='last'):
        super(LSTMTextClassifier, self).__init__()
        self.aggregate = aggregate
        # Embedding layer with pretrained word vectors
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix).to(device))
        self.embedding.weight.requires_grad = False # freeze the embeddings
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        
        # Softmax Layer
        self.softmax_layer = nn.Sequential(nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim),
        nn.Softmax(dim=1))
        
    def forward(self, text, text_lengths):
        # Embed the input
        embedded = self.embedding(text)
        
        #Pack the embedded sequences to handle variable-length sequences
        packed_embedded = pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted=False)

        # Pass through the LSTM layer
        packed_output, (hidden, cell) = self.lstm(packed_embedded)

        # Unpack the packed sequences
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        if(self.aggregate == 'last'):
            # Use the final hidden state as the representation for the sentence
            hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1) if self.lstm.bidirectional else hidden[-1, :, :]
            # Use max pooling to aggregate as the representation for the sentence
        elif(self.aggregate == 'max'):
            hidden, _ = torch.max(output, dim = 1)
            # Use average pooling to aggregate as the representation for the sentence
        elif(self.aggregate == 'average'):
            hidden = torch.mean(output, dim = 1)
        
        # Pass through the softmax layer
        output = self.softmax_layer(hidden)
        return output

### Declare model parameters

In [26]:
embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float)
hidden_dim = 256
output_dim = 5
vocab_size = len(word2vec.index_to_key)
embedding_dim = 300
num_layer = 1
device = torch.device('cuda')

### Train function

In [27]:
def train(model, train_dataloader, valid_dataloader, optimizer, loss_fn, device, num_epochs=50):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        # ======== training phase ==========
        train_loss = 0.0
        for batch in train_dataloader:
            text, labels, text_lengths = batch
            text, labels, text_lengths  = torch.tensor(text).to(device), labels.to(device), text_lengths

            optimizer.zero_grad() # clear gradients

            output = model(text, text_lengths)

            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        
        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_dataloader)
        # ====================================


        # ========== validation phase =========
        model.eval()
        valid_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in valid_dataloader:
                text, labels, text_lengths = batch
                text, labels, text_lengths  = text.to(device), labels.to(device), text_lengths

                output = model(text, text_lengths)
                loss = loss_fn(output, labels)
                valid_loss += loss.item()

                _, predicted = torch.max(output, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate average validation loss and accuracy for the epoch
        avg_valid_loss = valid_loss / len(valid_dataloader)
        accuracy = 100 * (correct / total)
        # =======================================
        print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%')

    print('Training complete.')


### Test function

In [28]:
def test(model, test_dataloader, device, loss_fn):
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            text, labels, text_lengths = batch
            text, labels, text_lengths  = text.to(device), labels.to(device), text_lengths

            output = model(text, text_lengths)
            loss = loss_fn(output, labels)
            valid_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    avg_valid_loss = valid_loss / len(test_dataloader)
    accuracy = 100 * (correct / total)

    return avg_valid_loss, accuracy

# Train and test model with aggregation as taking representation of last word

In [29]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [30]:
lstm_classifier_last = LSTMTextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim,num_layer,False, device, aggregate='last')
lr = 0.001
optimizer = optim.Adam(lstm_classifier_last.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

C:\Users\mdzak\AppData\Local\Temp\ipykernel_22820\1319915531.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix).to(device))


### Train model

In [31]:
train(lstm_classifier_last, train_dataloader, development_dataloader, optimizer, loss_fn, device, num_epochs=50)

C:\Users\mdzak\AppData\Local\Temp\ipykernel_22820\2511409481.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text, labels, text_lengths  = torch.tensor(text).to(device), labels.to(device), text_lengths


Epoch 1/50, Train Loss: 1.4756, Validation Loss: 1.2941, Accuracy: 59.60%
Epoch 2/50, Train Loss: 1.4863, Validation Loss: 1.6620, Accuracy: 22.40%
Epoch 3/50, Train Loss: 1.3804, Validation Loss: 1.4556, Accuracy: 44.00%
Epoch 4/50, Train Loss: 1.3101, Validation Loss: 1.2174, Accuracy: 67.80%
Epoch 5/50, Train Loss: 1.2108, Validation Loss: 1.1993, Accuracy: 71.20%
Epoch 6/50, Train Loss: 1.3215, Validation Loss: 1.2460, Accuracy: 66.40%
Epoch 7/50, Train Loss: 1.3590, Validation Loss: 1.2532, Accuracy: 66.00%
Epoch 8/50, Train Loss: 1.4442, Validation Loss: 1.4604, Accuracy: 44.40%
Epoch 9/50, Train Loss: 1.4614, Validation Loss: 1.4291, Accuracy: 45.80%
Epoch 10/50, Train Loss: 1.2585, Validation Loss: 1.1811, Accuracy: 73.00%
Epoch 11/50, Train Loss: 1.1709, Validation Loss: 1.1828, Accuracy: 71.60%
Epoch 12/50, Train Loss: 1.1253, Validation Loss: 1.1152, Accuracy: 79.20%
Epoch 13/50, Train Loss: 1.1038, Validation Loss: 1.1284, Accuracy: 77.60%
Epoch 14/50, Train Loss: 1.1112, V

### Get test accuracy and loss for test datatset

In [32]:
avg_valid_loss, accuracy = test(lstm_classifier_last, test_dataloader, device, loss_fn)

print("Avg test loss:", avg_valid_loss)
print("Accuracy:", accuracy)

Avg test loss: 1.025850772857666
Accuracy: 88.0


# Train and test model with aggregation as max pooling

In [33]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [34]:
lstm_classifier_max = LSTMTextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim,num_layer,False, device, aggregate='max')
lr = 0.001
optimizer = optim.Adam(lstm_classifier_max.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

C:\Users\mdzak\AppData\Local\Temp\ipykernel_22820\1319915531.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix).to(device))


In [35]:
train(lstm_classifier_max, train_dataloader, development_dataloader, optimizer, loss_fn, device, num_epochs=50)

C:\Users\mdzak\AppData\Local\Temp\ipykernel_22820\2511409481.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text, labels, text_lengths  = torch.tensor(text).to(device), labels.to(device), text_lengths


Epoch 1/50, Train Loss: 1.3659, Validation Loss: 1.2386, Accuracy: 65.80%
Epoch 2/50, Train Loss: 1.1730, Validation Loss: 1.1676, Accuracy: 73.00%
Epoch 3/50, Train Loss: 1.1332, Validation Loss: 1.1111, Accuracy: 79.40%
Epoch 4/50, Train Loss: 1.0952, Validation Loss: 1.0951, Accuracy: 81.40%
Epoch 5/50, Train Loss: 1.1001, Validation Loss: 1.0866, Accuracy: 81.60%
Epoch 6/50, Train Loss: 1.0712, Validation Loss: 1.0851, Accuracy: 82.60%
Epoch 7/50, Train Loss: 1.0763, Validation Loss: 1.0834, Accuracy: 82.20%
Epoch 8/50, Train Loss: 1.0897, Validation Loss: 1.1452, Accuracy: 75.60%
Epoch 9/50, Train Loss: 1.0760, Validation Loss: 1.0935, Accuracy: 81.20%
Epoch 10/50, Train Loss: 1.0509, Validation Loss: 1.0617, Accuracy: 84.40%
Epoch 11/50, Train Loss: 1.0396, Validation Loss: 1.0707, Accuracy: 83.20%
Epoch 12/50, Train Loss: 1.0343, Validation Loss: 1.0601, Accuracy: 84.40%
Epoch 13/50, Train Loss: 1.0282, Validation Loss: 1.0760, Accuracy: 82.20%
Epoch 14/50, Train Loss: 1.0232, V

In [36]:
avg_valid_loss, accuracy = test(lstm_classifier_max, test_dataloader, device, loss_fn)

print("Avg test loss:", avg_valid_loss)
print("Accuracy:", accuracy)

Avg test loss: 1.0048667378723621
Accuracy: 89.8


# Train and test model with aggregation as average pooling

In [37]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [38]:
lstm_classifier_avg = LSTMTextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim,num_layer,False, device, aggregate='average')
lr = 0.001
optimizer = optim.Adam(lstm_classifier_avg.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

C:\Users\mdzak\AppData\Local\Temp\ipykernel_22820\1319915531.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix).to(device))


In [39]:
train(lstm_classifier_avg, train_dataloader, development_dataloader, optimizer, loss_fn, device, num_epochs=50)

C:\Users\mdzak\AppData\Local\Temp\ipykernel_22820\2511409481.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text, labels, text_lengths  = torch.tensor(text).to(device), labels.to(device), text_lengths


Epoch 1/50, Train Loss: 1.4364, Validation Loss: 1.3318, Accuracy: 55.60%
Epoch 2/50, Train Loss: 1.2499, Validation Loss: 1.2231, Accuracy: 69.80%
Epoch 3/50, Train Loss: 1.2092, Validation Loss: 1.2106, Accuracy: 69.80%
Epoch 4/50, Train Loss: 1.1742, Validation Loss: 1.1932, Accuracy: 72.00%
Epoch 5/50, Train Loss: 1.1945, Validation Loss: 1.1941, Accuracy: 71.80%
Epoch 6/50, Train Loss: 1.1763, Validation Loss: 1.2049, Accuracy: 70.40%
Epoch 7/50, Train Loss: 1.1402, Validation Loss: 1.1524, Accuracy: 76.00%
Epoch 8/50, Train Loss: 1.1429, Validation Loss: 1.1500, Accuracy: 76.20%
Epoch 9/50, Train Loss: 1.1614, Validation Loss: 1.2965, Accuracy: 60.60%
Epoch 10/50, Train Loss: 1.2201, Validation Loss: 1.1670, Accuracy: 74.20%
Epoch 11/50, Train Loss: 1.1546, Validation Loss: 1.1529, Accuracy: 74.80%
Epoch 12/50, Train Loss: 1.1172, Validation Loss: 1.1074, Accuracy: 80.20%
Epoch 13/50, Train Loss: 1.1255, Validation Loss: 1.1157, Accuracy: 79.80%
Epoch 14/50, Train Loss: 1.0943, V

In [40]:
avg_valid_loss, accuracy = test(lstm_classifier_avg, test_dataloader, device, loss_fn)

print("Avg test loss:", avg_valid_loss)
print("Accuracy:", accuracy)

Avg test loss: 1.04337877035141
Accuracy: 86.2
